# 第 7 章：计算图与图论（Computational Graph & Graph Theory）

本章关注：

- 如何用 DAG（有向无环图）表示模型
- graph rewrite / fusion 的数学抽象
- 如何基于图结构做 scheduling / partitioning


## 7.1 模型即计算图（DAG）

现代深度学习模型在推理框架/编译器内部都表示为 **有向无环图 DAG（Directed Acyclic Graph）**。

- **节点（Node）**：算子（Op），例如 MatMul、Add、GELU、LayerNorm、Conv、Transpose …
- **边（Edge）**：张量的数据流，表示谁的输出被谁作为输入使用

由于图是无环的（没有 op 依赖未来的结果），所以可以：

- 对图进行 **拓扑排序（topological sort）** 得到合法执行顺序
- 在图上做依赖分析、内存规划、并行调度等各种优化

下面用 PyTorch FX 展示一个真实 IR（中间表示）的例子。

In [26]:
%pip install tabulate

import torch
import torch.nn as nn
import torch.fx as fx

class MyBlock(nn.Module):
    def __init__(self, in_features, hidden):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden, 1)

    def forward(self, x):
        # y = fc2(relu(fc1(x)))
        return self.fc2(self.relu(self.fc1(x)))

model = MyBlock(16, 32)
gm = fx.symbolic_trace(model)

print("\n=== Tabular View ===")
gm.graph.print_tabular()


Note: you may need to restart the kernel to use updated packages.

=== Tabular View ===
opcode       name    target    args     kwargs
-----------  ------  --------  -------  --------
placeholder  x       x         ()       {}
call_module  fc1     fc1       (x,)     {}
call_module  relu    relu      (fc1,)   {}
call_module  fc2     fc2       (relu,)  {}
output       output  output    (fc2,)   {}


**说明：**

- `gm` 是一个 `GraphModule`，内部的 `gm.graph` 就是 FX 的 **IR（Intermediate Representation，中间表示）**
- 每个 `node` 是一个算子（`call_module`, `placeholder`, `output` 等）
- `x → fc1 → relu → fc2 → output` 构成一条计算链，这本质上就是一个 **DAG**

在 TensorRT / XLA / TVM 里，IR 结构更复杂，但本质是同一类东西：  
**模型被抽象为“图 + 节点属性 + 边上的张量信息”**。

## 7.2 图重写（Graph Rewrite / Transform）

图重写的目标：

> 在保持 **输入输出语义完全一致** 的前提下，通过等价替换获得更高推理效率。

常见操作包括：

- **算子融合（Operator Fusion）**：如 MatMul + Bias + GELU → FusedMatMulBiasGelu
- **常量折叠（Constant Folding）**：所有只依赖常量的子图在编译期算掉
- **消除冗余（Elimination）**：合并/消除多余的 reshape / transpose / cast 等

这一节先用 FX 做一个实际可运行的 **Linear + ReLU 融合** 示例。


In [27]:
class LinearReLU(nn.Module):
    """将 Linear + ReLU 融合成一个模块，用于示例。"""
    def __init__(self, linear: nn.Linear):
        super().__init__()
        # 复用原来的权重和 bias
        self.linear = linear
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.linear(x))


def fuse_linear_relu(gm: fx.GraphModule) -> fx.GraphModule:
    graph = gm.graph
    modules = dict(gm.named_modules())

    print("=== Before Fused FX Graph ===")
    graph.print_tabular()

    for node in list(graph.nodes):
        # 找 ReLU
        if node.op != "call_module":
            continue
        if not isinstance(modules.get(node.target, None), nn.ReLU):
            continue

        relu_node = node
        prev = relu_node.args[0]

        # 要求前一个是 Linear
        if prev.op != "call_module":
            continue
        if not isinstance(modules.get(prev.target, None), nn.Linear):
            continue

        linear_node = prev

        # 安全检查：Linear 的输出只被 ReLU 用过一次
        if len(linear_node.users) != 1:
            # 如果 Linear 还有别的用户，就先别 fuse，避免改错语义
            continue

        linear_module = modules[linear_node.target]

        fused = LinearReLU(linear_module)
        fused_name = linear_node.target + "_relu_fused"
        gm.add_submodule(fused_name, fused)

        # 新节点用 linear 的输入作为输入
        with graph.inserting_after(linear_node):
            new_node = graph.call_module(fused_name, args=linear_node.args)

        # 所有用 ReLU 输出的地方改为用 fused
        relu_node.replace_all_uses_with(new_node)

        # 删除 ReLU 节点和 Linear 节点 —— 此时图里只剩 fused
        graph.erase_node(relu_node)
        graph.erase_node(linear_node)

    graph.lint()
    gm.recompile()
    return gm
    
# 对上一节中的 gm 做融合
model = MyBlock(16, 32)
gm = fx.symbolic_trace(model)

gm_fused = fuse_linear_relu(gm)
print("=== Fused FX Graph ===")
print(gm_fused.graph)

gm.graph.print_tabular()

# 验证融合前后结果相同
x = torch.randn(4, 16)
y_ref = model(x)
y_fused = gm_fused(x)

# should be very close to 0
print((y_ref - y_fused).abs().max())


=== Before Fused FX Graph ===
opcode       name    target    args     kwargs
-----------  ------  --------  -------  --------
placeholder  x       x         ()       {}
call_module  fc1     fc1       (x,)     {}
call_module  relu    relu      (fc1,)   {}
call_module  fc2     fc2       (relu,)  {}
output       output  output    (fc2,)   {}
=== Fused FX Graph ===
graph():
    %x : [num_users=1] = placeholder[target=x]
    %fc1_relu_fused : [num_users=1] = call_module[target=fc1_relu_fused](args = (%x,), kwargs = {})
    %fc2 : [num_users=1] = call_module[target=fc2](args = (%fc1_relu_fused,), kwargs = {})
    return fc2
opcode       name            target          args               kwargs
-----------  --------------  --------------  -----------------  --------
placeholder  x               x               ()                 {}
call_module  fc1_relu_fused  fc1_relu_fused  (x,)               {}
call_module  fc2             fc2             (fc1_relu_fused,)  {}
output       output          

**工程含义：**

- 我们在 **IR 上进行 pattern 匹配和替换** → 这就是图重写
- 真实框架中的 fusion pass 本质也是：  
  找到 `P` 形式的子图，用更高效的 `R` 形式子图替换

在车端推理（batch=1）场景下：

- Kernel launch 开销与 DRAM 往返代价非常显著
- 通过 fusion，减少中间 tensor 写回内存、减少 kernel 数量，是性能优化的关键步骤


### 7.2.2 ONNX 风格 IR：MatMul + Add Fusion 伪代码示例

真实的 ONNX Runtime / TensorRT / TVM 都有自己的 IR。  
下面构造一个极简 Graph IR，模仿 ONNX 风格。然后构造一个 `MatMul → Add → Relu` 的小子图，模拟从 PyTorch 导出的 ONNX 子图，然后写一个简化的 **MatMul + Add → FusedMatMulBias** 的融合 pass，逻辑与真实框架类似：


In [28]:
from dataclasses import dataclass, field
from typing import List, Dict, Any

@dataclass
class Node:
    name: str
    op_type: str
    inputs: List[str]
    outputs: List[str]
    attrs: Dict[str, Any] = field(default_factory=dict)

@dataclass
class GraphIR:
    nodes: List[Node]
    initializers: Dict[str, Any]  # 常量 tensor
    inputs: List[str]
    outputs: List[str]

graph = GraphIR(
    nodes=[
        Node("matmul1", "MatMul", ["X", "W"], ["Z"]),
        Node("add1", "Add", ["Z", "B"], ["Z2"]),
        Node("relu1", "Relu", ["Z2"], ["Y"]),
    ],
    initializers={
        "W": "...权重 tensor...",
        "B": "...bias tensor...",
    },
    inputs=["X"],
    outputs=["Y"],
)

graph


GraphIR(nodes=[Node(name='matmul1', op_type='MatMul', inputs=['X', 'W'], outputs=['Z'], attrs={}), Node(name='add1', op_type='Add', inputs=['Z', 'B'], outputs=['Z2'], attrs={}), Node(name='relu1', op_type='Relu', inputs=['Z2'], outputs=['Y'], attrs={})], initializers={'W': '...权重 tensor...', 'B': '...bias tensor...'}, inputs=['X'], outputs=['Y'])

In [29]:
def fuse_matmul_add(graph: GraphIR) -> GraphIR:
    """将 MatMul + Add (bias 为常量) 融合为单个 FusedMatMulBias 节点。"""
    new_nodes: List[Node] = []
    for node in graph.nodes:
        if node.op_type == "MatMul":
            matmul = node
            matmul_out = matmul.outputs[0]

            add_node = None
            for n in graph.nodes:
                if n.op_type == "Add" and matmul_out in n.inputs:
                    add_node = n
                    break

            if add_node is not None:
                # 另一个输入视作 bias
                bias_name = [i for i in add_node.inputs if i != matmul_out][0]
                if bias_name in graph.initializers:
                    fused = Node(
                        name=matmul.name + "_fused",
                        op_type="FusedMatMulBias",
                        inputs=[matmul.inputs[0], matmul.inputs[1], bias_name],
                        outputs=add_node.outputs,
                    )
                    new_nodes.append(fused)
                    continue  # 不再保留原 MatMul / Add

        # 默认保留
        new_nodes.append(node)

    graph.nodes = new_nodes
    return graph

fused_graph = fuse_matmul_add(graph)
fused_graph


GraphIR(nodes=[Node(name='matmul1_fused', op_type='FusedMatMulBias', inputs=['X', 'W', 'B'], outputs=['Z2'], attrs={}), Node(name='add1', op_type='Add', inputs=['Z', 'B'], outputs=['Z2'], attrs={}), Node(name='relu1', op_type='Relu', inputs=['Z2'], outputs=['Y'], attrs={})], initializers={'W': '...权重 tensor...', 'B': '...bias tensor...'}, inputs=['X'], outputs=['Y'])

**说明：**

- 我们在 `GraphIR` 上做 pattern 匹配和替换，这就是 ONNX / TensorRT 里常见的 fusion pass 思路
- 实际工程中会：
  - 处理更多类型（Conv+BN+ReLU、LayerNorm+Add 等）
  - 使用更复杂的 pattern 表达（子图匹配）
  - 引入 cost model，根据形状/设备/数据类型决策是否值得融合

### 7.2.3 Constant Folding（常量折叠）

如果某个节点的所有输入均为常量，则可以在编译期直接算出结果，并将输出标记为新的常量，删除该节点。
**工程意义：**

- 编译期提前消掉常量路径，推理时可以少跑很多节点
- 对于大模型（尤其是含有大量常量子图的结构），constant folding 可以显著减少图规模


In [30]:
import numpy as np

def constant_folding(graph: GraphIR) -> GraphIR:
    """对 GraphIR 执行简化版常量折叠：这里只处理 Add 节点。"""
    changed = True
    while changed:
        changed = False
        new_nodes: List[Node] = []
        for node in graph.nodes:
            if node.op_type == "Add":
                inp0, inp1 = node.inputs
                if inp0 in graph.initializers and inp1 in graph.initializers:
                    v0 = np.array(graph.initializers[inp0])
                    v1 = np.array(graph.initializers[inp1])
                    out_name = node.outputs[0]
                    graph.initializers[out_name] = (v0 + v1)
                    changed = True
                    # 这个 Add 节点可以被删除，不加入 new_nodes
                    continue
            new_nodes.append(node)
        graph.nodes = new_nodes
    return graph

# 构造一个简单示例
cf_graph = GraphIR(
    nodes=[
        Node("add_const", "Add", ["C1", "C2"], ["C3"]),
        Node("add_mixed", "Add", ["X", "C3"], ["Y"]),
    ],
    initializers={
        "C1": np.array([1, 2, 3]),
        "C2": np.array([4, 5, 6]),
    },
    inputs=["X"],
    outputs=["Y"],
)

print("Before folding:", cf_graph.initializers.keys(), [n.name for n in cf_graph.nodes])
cf_graph = constant_folding(cf_graph)
print("After folding:", cf_graph.initializers.keys(), [n.name for n in cf_graph.nodes])
print("C3 value:", cf_graph.initializers["C3"])


Before folding: dict_keys(['C1', 'C2']) ['add_const', 'add_mixed']
After folding: dict_keys(['C1', 'C2', 'C3']) ['add_mixed']
C3 value: [5 7 9]


## 7.3 图划分与多设备并行（Partitioning & Parallelism）

在车载/边缘设备中，通常存在多种计算单元：

- 多核 CPU
- GPU
- NPU / DSP（自研加速器）
- 有时还有特定功能单元（ISP、编码器等）

需要根据 **图结构 + 设备特性** 决定：

- 哪个子图放在 CPU 上
- 哪个子图放在 GPU / NPU 上
- 如何减少跨设备的数据拷贝（边 cut）

数学上是一个 **图划分（graph partitioning）** 问题，一般是 NP-hard，只能用启发式方法/代价模型。

### 7.3.1 简单 Partition 示例：Conv/MatMul → NPU，其余 → CPU

In [31]:
def simple_partition(graph: GraphIR):
    """一个非常简化的 partition 示例：Conv/MatMul 放 NPU，其余放 CPU。"""
    cpu_nodes = []
    npu_nodes = []

    for node in graph.nodes:
        if node.op_type in ["Conv", "MatMul", "FusedMatMulBias"]:
            npu_nodes.append(node)
        else:
            cpu_nodes.append(node)

    return cpu_nodes, npu_nodes

# 这里复用前面的 fused_graph 作为示例
cpu_subgraph, npu_subgraph = simple_partition(fused_graph)

print("CPU 子图节点:", [n.name for n in cpu_subgraph])
print("NPU 子图节点:", [n.name for n in npu_subgraph])

CPU 子图节点: ['add1', 'relu1']
NPU 子图节点: ['matmul1_fused']


真实系统中还会考虑：

- 设备是否支持该算子、数据类型（如 INT8 / FP16）
- 各设备的算力、带宽、latency
- 跨设备拷贝的代价（PCIe / 内存总线）
- 内存容量限制

但本质思路都是在 **IR 的图结构上进行子图划分**。

## 7.4 Liveness 分析与内存规划（Memory Planning）

边缘设备/车载 ECU 常见瓶颈：

- DRAM 容量有限
- 带宽有限
- activation 远大于参数大小

因此需要：

- 分析每个 tensor 的**生命周期（liveness interval）**
- 通过 **区间图着色（interval graph coloring）** 方式复用 buffer
- 降低峰值内存占用（peak memory）

### 7.4.1 构造一个简单执行计划并做 Liveness 分析

In [32]:
class OpSimple:
    def __init__(self, name, inputs, outputs):
        self.name = name
        self.inputs = inputs
        self.outputs = outputs

    def __repr__(self):
        return f"OpSimple(name={self.name}, inputs={self.inputs}, outputs={self.outputs})"


ops = [
    OpSimple("op1", ["x"], ["t1"]),
    OpSimple("op2", ["t1"], ["t2"]),
    OpSimple("op3", ["t2"], ["t3"]),
    OpSimple("op4", ["t1"], ["t4"]),
    OpSimple("op5", ["t3", "t4"], ["y"]),
]

for i, op in enumerate(ops):
    print(i, op)

def compute_liveness(ops):
    """计算每个 tensor 的 [start, end] 生命周期区间。"""
    first_use = {}
    last_use = {}

    for idx, op in enumerate(ops):
        # 输出：birth
        for out in op.outputs:
            if out not in first_use:
                first_use[out] = idx
            last_use[out] = idx

        # 输入：被使用
        for inp in op.inputs:
            if inp not in first_use:
                # 认为图输入在 0 之前就存在
                first_use[inp] = 0
            last_use[inp] = idx

    intervals = {t: (first_use[t], last_use[t]) for t in first_use}
    return intervals

intervals = compute_liveness(ops)
print("Tensor liveness intervals:")
for t, (s, e) in intervals.items():
    print(f"  {t}: [{s}, {e}]")

0 OpSimple(name=op1, inputs=['x'], outputs=['t1'])
1 OpSimple(name=op2, inputs=['t1'], outputs=['t2'])
2 OpSimple(name=op3, inputs=['t2'], outputs=['t3'])
3 OpSimple(name=op4, inputs=['t1'], outputs=['t4'])
4 OpSimple(name=op5, inputs=['t3', 't4'], outputs=['y'])
Tensor liveness intervals:
  t1: [0, 3]
  x: [0, 0]
  t2: [1, 2]
  t3: [2, 4]
  t4: [3, 4]
  y: [4, 4]


### 7.4.2 区间图着色（Interval Coloring）实现 Buffer 复用


In [33]:
def allocate_buffers(intervals):
    """使用最简单的 greedy 算法为每个 tensor 分配 buffer ID。"""
    # 按照开始时间排序
    items = sorted(intervals.items(), key=lambda x: x[1][0])
    buffers = []  # 每个 buffer 当前的 end 时间
    assignment = {}

    for tensor, (start, end) in items:
        assigned = False
        for buf_id, buf_end in enumerate(buffers):
            if start > buf_end:
                # 该 buffer 已空闲，可复用
                buffers[buf_id] = end
                assignment[tensor] = buf_id
                assigned = True
                break
        if not assigned:
            # 需要新建一个 buffer
            new_id = len(buffers)
            buffers.append(end)
            assignment[tensor] = new_id

    return assignment, len(buffers)

buf_assign, num_buf = allocate_buffers(intervals)
print("Buffer assignment:")
for t, b in buf_assign.items():
    print(f"  tensor {t} -> buffer {b}")
print("Total buffers used:", num_buf)

Buffer assignment:
  tensor t1 -> buffer 0
  tensor x -> buffer 1
  tensor t2 -> buffer 1
  tensor t3 -> buffer 2
  tensor t4 -> buffer 1
  tensor y -> buffer 0
Total buffers used: 3


**含义：**

- 多个 tensor 在时间上“错峰”使用同一个 buffer，从而降低了内存峰值
- 在真实框架中还会考虑：
  - tensor 大小不同
  - 对齐（alignment）
  - 不同设备（CPU/GPU/NPU）各自的内存池
  - activation checkpointing / recompute 策略

但其数学内核就是：**区间图着色问题**。



⭐ 1. 工程中是否手写区间着色算法？
**不需要**。

**内存规划完全由编译器自动完成。**

几乎所有现代推理编译器 / runtime 都内置了内存规划器：
| 框架                           | 内存规划组件                             | 是否使用图着色思想                              |
| ---------------------------- | ---------------------------------- | -------------------------------------- |
| **TensorRT**                 | TensorRT Memory Planner            | ✔️ 是，buffer reuse + lifetime inference |
| **TensorFlow Lite**          | Arena Planner                      | ✔️ 是（interval reuse）                   |
| **ONNX Runtime**             | Memory Planner / Custom Allocators | ✔️                                     |
| **XLA**                      | Global Buffer Assignment           | ✔️（区间 + interference graph coloring）   |
| **TVM**                      | Memory Plan Pass                   | ✔️（reuse + sharing + pool allocator）   |
| **PyTorch 2.0 AOT Autograd** | Memory Planner                     | ✔️                                     |
| **AITemplate**               | Memory Planning Pass               | ✔️（GPU static memory planning）         |
| **CoreML**                   | Storage Fuse Optimization          | ✔️                                     |
所以在工业界：

工程师不会手工分配 tensor buffer，所有内存复用都通过 compile-time pass 自动生成。

---
2. 工程师需要写的是“策略”，不是着色算法本身
内存规划分为两个层次：
（1）算法层（内核级）— 自动完成

包括：

- liveness interval 计算

- interference graph 构建

- interval graph coloring

- global memory assignment

- pool / arena / chunking

这些都由框架本身实现，工程师不手写。
例如 TensorRT 内存规划：
- Build dependency DAG
- Compute tensor liveness
- Build interference graph
- Allocate global memory pool
- Assign offsets for each tensor


（2）策略层（policy level）— 工程师需要参与

你需要提供/调整：

- 哪些 tensor 强制不复用（如 debug dump、persistent memory）

- 哪些 tensor 允许 in-place

- 哪些 op 可以 in-place 推导（Add、ReLU、BiasAdd 等）

- 不同 device（CPU/NPU/GPU）是否共用 memory pool

- 特殊 tensor 是否 pin memory / lock memory

- 特定硬件（NPU）是否需要 alignment=64/128/256

- 需要的 buffer chunk size（如 quantization scratch buffer）

- mixed precision 是否改变 tensor 大小

- dynamic shape 时是否使用 cheapest-fit / first-fit / dynamic pool

你写的是 “规则”，由编译器根据规则生成最终 layout。

（3） 优化角度就是算出一个 tensor buffer 的最小化分配策略，满足所有 tensor 的内存需求

原则上：是的，内存规划器的目标就是最小化峰值内存。

但实际工程上会用启发式策略（近似最优），不是严格全局最优。

为什么？
因为严格求最优是 NP-hard（图着色问题）。

实际框架用的方法：

- Greedy first-fit / best-fit

- Interval coloring + heuristics

- Chunk pool / arena allocator

- Buffer coalescing

- In-place propagation

这些方法很快（O(n log n) ~ O(n)），且在图形化 workload（DNN）中通常接近最优。

（4） 工程实际：Tensor buffer 就是

- CPU RAM buffer

- GPU VRAM buffer

- NPU SRAM/DRAM buffer

- 或 device-specific unified memory

它们都是：

实际的物理内存池里的一个连续区间（offset + size）。

框架会把每个 tensor 输出分配到：

global_memory_pool[offset : offset + size]

并确保：

- 活跃区间不重叠

- 对齐（alignment）满足硬件要求

- 不同设备可能有不同 memory pool

（5）. 在实际工程中，你会做什么？

你做的是 Runtime Inference / Deployment Engineer，所以你会：

- 理解 IR + liveness + memory planner 的逻辑
     -  某个 tensor 为什么不能复用 buffer

     - 内存峰值为什么上不去（瓶颈 subgraph）

     - 哪个 pass 把某些 in-place 机会给消掉了

     -  NPU 的 memory layout 改变是否导致 buffer 分裂

- 写策略（policy rules）
     - 某些算子（BatchNorm、LayerNorm）可以 in-place

     - 某些算子必须分配新 buffer（因为 hardware hazard）

     - 不同 device 要分离 memory pool 或共用

     -  增加特殊 alignment（比如 128 bytes）以符合 NPU DMA 要求

     - 动态 shape 模型用 dynamic memory pool 而不是 static graph plan

- 调试和验证

     - 查看内存分配 trace（很多 runtime 会 output memory timeline）

     - 优化 out-of-memory 场景

     - 减少 peak memory，使大模型跑上车载设备

     - 在 low-memory 设备上通过：
          - activation recompute
          - chunking
          - streaming

    - 降低 footprint

- 协助实现（或改进）某些 pass
     - “linear fusion” pass 后 op 的 size 变了，会影响 memory planning

     - 优化 transpose folding，会减少中间 tensor 数量

     - quantization pass 改变 tensor type 和 size，影响 buffer reuse

     - operator placement（CPU/GPU/NPU）影响 memory pool 结构



## 7.5 工程级小结：图论如何支撑 Runtime Inference 优化

| 图论概念 | 工程价值 | 对 Runtime Inference / Edge Deployment 的作用 |
|---------|----------|-----------------------------------------------|
| DAG（有向无环图） | 定义执行顺序与依赖关系 | 支撑拓扑排序、调度、分析依赖、可视化性能瓶颈 |
| 图重写（graph rewrite） | fusion/folding/elimination | 减少 kernel 数量、减少 DRAM 往返、清理无用子图，提升吞吐/降低延迟 |
| 图划分（graph partitioning） | 多设备并行与负载均衡 | 决定子图放在哪个引擎（CPU/GPU/NPU），减少跨设备复制 |
| Liveness 分析 & 区间图着色 | 内存复用与 buffer 规划 | 降低峰值内存，使大模型 fit 进受限的车载/边缘设备 |

对实际的工作来说：

- **IR 阅读能力**：看得懂内部图、能快速定位性能瓶颈子图
- **图重写/融合能力**：可以和编译器团队一起设计 pass，提高算子覆盖和效率
- **Partition/Placement 直觉**：理解为什么某些 op 被调度在特定引擎上，以及如何优化
- **内存规划意识**：在设计模型/优化 pipeline 时主动考虑内存峰值，而不是事后救火